## An Analysis of the 2024 Syracuse Mets
In this analysis we're going to be looking at basic stats collected from Baseball Refernce and new advanced stats scraped from Baseball Savant's minor league stat cast database.
I'm going to focus on hitters with over 100 plate appearences so we can get an accurate understanding of players who have a decent sample size of play in AAA. I'll also be looking at players under the age of 27 since the Mets as an organization have a large focus on their young prospects that can help the team win in the near future.

In [1]:
import pandas as pd
from utils import *

pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', None)

# Select Mets players below 27 and with over 100 plate appearences
completeMetsAAAStats = completeMetsAAAStats[(completeMetsAAAStats['PA'] > 100) & (completeMetsAAAStats['Age'] <= 26)]

## Initial Analysis / Basic Slash Line

To start our analysis, I'm going to look at the most traditional way we look at hitters: the slash line. Now I know OPS is not included in a traditional slash line but I put it because I believe it's one of the best non-advanced stats to display a hitter's overall performance.

With our criteria stated above, we're given 5 players on the team that meet these requirements: Brett Baty, Mark Vientos, Luisangel Acuna, Thomas Rhyland, and Drew Gilbert.

Looking at these stats I can see two standouts, Brett Baty and Mark Vientos. I say these are our standouts because they're hitting above .800 OPS demonstrating an ability to hit for power and get on base, at least on a surface level. 

Let's take a look at how these players stack up versus AAA league averages.

In [2]:
# Select the basic statline stats and basic info I want to see for each player: Player, Age, G, PA, AB, BA, OBP, SLG, OPS
print(completeMetsAAAStats[defaultInfo + slashLine].sort_values(by='OPS', ascending=False))

                             Player   Age    G   PA   AB     BA    OBP    SLG    OPS
3                       Baty, Brett  24.0   56  246  211  0.261  0.358  0.536  0.885
27                    Vientos, Mark  24.0   31  132  116  0.284  0.371  0.500  0.876
0        Acuï¿½a, Luisangel NYM #12  22.0  110  486  452  0.265  0.313  0.369  0.676
25                   Thomas, Rhylan  24.0   49  192  179  0.229  0.281  0.374  0.663
9   Gilbert, Drew MLB #81 / #NYM #3  23.0   26  111   97  0.216  0.315  0.309  0.577


## League Averages Comparison
<p>
In terms of batting average, Brett Baty sits at just above the league average. But if we continue to look down his stat line, he's anything but average. Baty's in the top half of the league for OBP which is incredibly valuable in today's game. You need base runners to score runs. However, this is not where his strangth is. His strength lies in his SLG/power. His slugging percentage is in the top quartile for the league, pretty handidly too. He's also in the top quartile for OPS. On a basic level, this shows his ability to hit for power while not compromising his ability to get on base. 

Next, I'll be looking at Mark Vientos. Although Mark Vientos has already been promoted to the Majors, it's important to look at the stats of players that have been called up and had success in the MLB. And I would definitely put Mark Vientos in this category with his incredible 2024 season. When looking at Vientos' batting average, it's clear he was having some level of success at the plate. He ranked in the upper quartile comfortably. Suprsingly, this is true for all of his stats except OBP where he still ranked in the 3rd quartile. Similar to Baty, Mark Vientos' slash line conveys his ability to hit for power but where he outperforms Baty is getting on base and getting on base through contact. However, Baty still does slightly have the edge in terms of power if we're basing this purely on slugging percentage. 

Now lets take a look at one of our top prospects: Luisangel Acuña. At first glance, his slash line is extremly underwhelming. For every stat except batting average, he ranks in the bottom quartile of the league. And even his batting average only falls into the third quartile. This tells me that despite the fact he can get on base fairly well through his hits, his overall ability to get on base as well as his power is lacking.

A lot of these observations for Acuña can made for Drew Gilbert as well thus far. They also fall into the bottom quartile for all slash line stats as well as OBP. However, with Drew Gilbert it's important to note his experience in AAA so far. He's barely had 100 plate appearences and more likely than not just needs more time to adjust to AAA. He previously put up decent numbers in the lower levels. We traded Rylan Thomas for Ryne Stanek so I won't focus on him too much.
</p>

In [3]:
# Clean and convert columns that are mistyped
columnsToConvert = ['OBP', 'SLG', 'BA']
leaguewideAAAStatsDf = convertColumnsToNumeric(leaguewideAAAStatsDf, columnsToConvert).copy()

# Calculate OPS for leaguewide data, this isn't a stat provided to us by Baseball Savant in the league data
leaguewideAAAStatsDf['OPS'] = leaguewideAAAStatsDf['OBP'] + leaguewideAAAStatsDf['SLG']

# Select all players in AAA with over 100 plate appearences
leaguewideAAAStatsDf = leaguewideAAAStatsDf[leaguewideAAAStatsDf['PA'] > 100]

# Get league mean, standard deviation, percentiles, and min/max values for BA, OBP, SLG, and OPS and concat the dataframes to one list for easy viewing. 
summary_df = pd.concat([
    leaguewideAAAStatsDf['BA'].describe(),
    leaguewideAAAStatsDf['OBP'].describe(),
    leaguewideAAAStatsDf['SLG'].describe(),
    leaguewideAAAStatsDf['OPS'].describe()
], axis=1)

print(summary_df)

               BA         OBP         SLG         OPS
count  471.000000  471.000000  471.000000  471.000000
mean     0.257062    0.347420    0.428486    0.775907
std      0.038070    0.042097    0.081750    0.112078
min      0.135000    0.204000    0.226000    0.431000
25%      0.234000    0.322000    0.374000    0.703000
50%      0.258000    0.346000    0.431000    0.780000
75%      0.280000    0.373000    0.480500    0.853500
max      0.401000    0.515000    0.678000    1.179000


## Actual Stats vs Expected Stats
Before looking at potential reasons why these player's stats look like this, it's important to see how they're peforming vs their expected stats. 

Let's examine Brett Baty. 
While it's true Brett Baty is outpeforming Luisangel Acuña on offense, his actual stats are lower than his expected stats in every category. This could be the case for a number of reasons and doesn't necessarily mean he's underperforming. Since Baty's batting average is lower than his expected output, this could suggest a number of possibilities. It is very possible he's just outright underperorming, but more likely than not it means he's getting very unlucky with some hard hit balls or hitting against some difficult defensive alignments. I say this because a .278 xBA is still great and if Baty continues to hit like this, we should see his actual batting average go up. I would give a similar analysis to his OBP vs xOBP, SLG vs xSLG, and wOBA vs xwOBA. If Baty continues the quality of his at bats and contact, his slash line should continue to trend up.

Next lets look at an interesting case: Mark Vientos. Despite the fact that he's doing better than Baty in every stat besides SLG, all of his expected stats are lower than Baty's. Since Vientos' actual stats are significantly higher than his expected, this could be indicative of a few things. It could mean he's getting lucky for the quality of his contact. Maybe he's getting a few lucky bounces or bloop hits that would raise his average. Or he could be potentially hitting home runs/extra-base hits that might not always carry out of the park or fall in for hits. While this might indicate his actual stats aren't sustainable, I peronsally believe these should be taken with a grain of salt. It's important to also look at the quality of contact and power a hitter is making to come to a complete conclusion, which we'll do later on. 

Let's move on to Luisangel Acuna. Similar to Vientos, he's outpeforming his expected stats except all of his actual and expected stats are lower than Vientos'. This shows me that if Vientos is getting lucky, then Acuna is getting nearly as lucky as seeing less of a payoff for it. His average is lower, he's getting on base less, and power is lacking. So the observations we made for Vientos' contact quality and base hit luck are same that I'd make here and really for all the prospects beneath Acuna as well. 

While I think these are important stats to note and observe, I don't know if they necessarily directly translate to any concrete predictiohs. While Baty's expected stats are better than Vientos', Baty has struggled at the major league level while Vientos has seen success this season. This is not to say ignore these stats, but they do not always paint a full picture.

In [4]:
print(completeMetsAAAStats[defaultInfo + actualVsEx + ['OPS']].sort_values(by='OPS', ascending=False))

                             Player   Age    G   PA   AB     BA    xBA    OBP   xOBP    SLG   xSLG   wOBA  xwOBA    OPS
3                       Baty, Brett  24.0   56  246  211  0.261  0.278  0.358  0.375  0.536  0.552  0.381  0.397  0.885
27                    Vientos, Mark  24.0   31  132  116  0.284  0.237  0.371  0.331  0.500  0.444  0.376  0.337  0.876
0        Acuï¿½a, Luisangel NYM #12  22.0  110  486  452  0.265  0.236  0.313  0.287  0.369  0.325  0.299  0.273  0.676
25                   Thomas, Rhylan  24.0   49  192  179  0.229  0.203  0.281  0.257  0.374  0.281  0.286  0.242  0.663
9   Gilbert, Drew MLB #81 / #NYM #3  23.0   26  111   97  0.216  0.189  0.315  0.291  0.309  0.282  0.286  0.264  0.577


## Luck? Or Skill?

Are they getting lucky? Or is quality contact being made? 

In [13]:
columnsToConvert = ['Barrel/PA%', 'Barrel/BBE%']
leaguewideAAAStatsDf = convertColumnsToNumeric(leaguewideAAAStatsDf, columnsToConvert)

print(completeMetsAAAStats[defaultInfo + contactQuality].sort_values(by='Barrel/BBE%', ascending=False))
print(completeMetsAAAStats[defaultInfo + powerQuality].sort_values(by='EV (MPH)', ascending=False))

summary_df = pd.concat([
    leaguewideAAAStatsDf['Barrel/BBE%'].describe(),
    leaguewideAAAStatsDf['Barrel/PA%'].describe()
], axis=1)

print(summary_df)

                             Player   Age    G   PA   AB  BIP  Barrels  Hard Hit%  EV (MPH)  LA (ï¿½)  Dist (ft)  Barrel/BBE%  Barrel/PA%
3                       Baty, Brett  24.0   56  246  211  164       26       46.3      88.5        13        167         15.9        10.6
27                    Vientos, Mark  24.0   31  132  116   78       10       47.4      91.8         5        151         12.8         7.6
9   Gilbert, Drew MLB #81 / #NYM #3  23.0   26  111   97   70        3       24.3      85.9        18        145          4.3         2.7
0        Acuï¿½a, Luisangel NYM #12  22.0  110  486  452  376       13       36.2      88.3         5        129          3.5         2.7
25                   Thomas, Rhylan  24.0   49  192  179  155        1       23.2      86.4        11        162          0.6         0.5
                             Player   Age    G   PA   AB  HR    SLG   xSLG    ISO  EV (MPH)  Adj. EV (MPH)  LA (ï¿½)  Dist (ft)  Hard Hit%
27                    Vientos, Ma